In [1]:
import torch_geometric.datasets as datasets

mutag = datasets.TUDataset(root='data', name='AIDS')

In [26]:
from gnnexplain.nn.gcn import GraphGCN
from torch_geometric.data import DataLoader

model = GraphGCN(38, 2, 64, 3)
train_loader = DataLoader(mutag, batch_size=256, shuffle=True)
val_loader = DataLoader(mutag, batch_size=64, shuffle=False)

/home/apluska/miniconda3/envs/gnnexplain/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [27]:
batch = train_loader.__iter__().__next__()

In [80]:
k_fold = 5; iteration = 0; name = 'MUTAG'

dataset = datasets.TUDataset(root='data', name=name)

n_val = len(dataset) // k_fold
val_data = dataset[iteration * n_val : (iteration + 1) * n_val]
train_data = dataset[:iteration * n_val] + dataset[(iteration + 1) * n_val:]

In [93]:
len(train_data.datasets[1])

MUTAG(151)

In [45]:
import torch

out = model(batch)
labels = torch.arange(model.num_classes, device=out.device)
sample_weights = torch.sum(batch.y.view(-1, 1) == labels.view(1, -1), dim=0).float()
sample_weights = model.num_classes * sample_weights / sample_weights.sum()
loss = model.loss(out / sample_weights, batch.y)

In [48]:
3 /  torch.tensor([1, 2, 3])

tensor([3.0000, 1.5000, 1.0000])

In [47]:
loss

tensor(0.6988, grad_fn=<NllLossBackward0>)

In [4]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch

loader = DataLoader(mutag, batch_size=40000, shuffle=False)
batch = loader.__iter__().__next__().cpu()
l = batch.to_data_list()
train, test = Batch.from_data_list(l[:150]), Batch.from_data_list(l[150:])

In [12]:
from gnnexplain.model.gtree import *
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)
opt = Optimizer(n_trials=1e3, max_ccp_alpha=1e-2, max_depth=10, lmb=1e-3)
expl = opt.optimize(train, model)

Best trial: 632. Best value: 0.863: 100%|██████████| 1000/1000.0 [03:13<00:00,  5.17it/s]


In [14]:
expl.accuracy(train)

0.84

In [15]:
expl.accuracy(test)

0.7631578947368421